# TiXI Exercise 2: Calculate Range of an Aircraft

In this exercise we will learn how to perform an analysis using the data read from a CPACS file that is a result of **EXACT** project.
The CPACS file has been downloaded from **Digital Hangar**. You can reach to the website by clicking on this link: https://www.digital-hangar.de/

The goal is to understand how performance parameters stored in CPACS can be accessed with **TiXI** and then applied to a simple performance model. Specifically, we will:

- Explore the structure of the CPACS file and identify key parameters  
- Use TiXI commands to read relevant data from CPACS file
- **Module 1:** Apply the **Breguet Range Equation** to estimate the aircraft’s actual cruise range  (Primary Exercise)
- **Module 2:** Study the **change in range** according to the change in fuel weight  (Optional Exercise)

By the end of this exercise, you will gain hands-on experience in:  

- Navigating XML data with TiXI  
- Performing a practical calculation directly linked to aircraft performance  
- Updating CPACS files with new results for downstream analysis  

This exercise aims to give you a first taste of how digital aircraft design workflows can be automated.


#### Note: Show solutions

To view and run the solutions, simply uncomment the # %load statement.

## Step 1: Initiate TiXI & Load Data
As always, the first thing we need to do is to import and initialize *tixi*:

In [ ]:
# import and initialize tixi
# ...

In [ ]:
# %load ./solutions/importTixi.py

Let us now open our example file located at `resources/D250-TF-2040.xml` that has been downloaded from Digital Hanger.

In [ ]:
# open the CPACS file with tixi
# ...

In [ ]:
# %load ./solutions/tixiOpenFile.py

## Step 2: Read Parameters

In this part of the exercise, we will perform an analysis based on the **Breguet Range Equation**.

### Breguet Range Equation

The range for a given amount of fuel is expressed as:

$
R = \frac{V}{\text{TSFC}} \cdot \frac{L}{D} \cdot 
\ln \left( 1 + \frac{W_{\text{fuel}}}{W_{PL} + W_0} \right)
$

Where:

- $V$ = Cruise Velocity  
- $\text{TSFC}$ = Thrust Specific Fuel Consumption
- $\frac{L}{D}$ = Lift-to-Drag Ratio  
- $W_{\text{fuel}}$ = Fuel Weight  
- $W_{PL}$ = Payload Weight  
- $W_0$ = Operating Empty Weight (OEW) or Dry Weight of Vehicle

To calculate the range, we first need to **read the relevant parameters** from the CPACS file using TiXI.  
Let’s check which values are already provided in the dataset before carrying out the calculation.

In [ ]:
# Known parameters for EXACT Turbofan Baseline aircraft
V_cruise = 231.3 # m/s
TSFC = 4.02e-4   # s⁻¹
L_D = 21

Now, we need to read the **fuel, payload, and overall empty weights** to perform the Range equation.

![Referencing the mass breakdown](resources/read_massBreakdown.png)

First, lets define the Xpaths of the weights:

In [ ]:
# Xpaths of the weight parametes
M_fuel_xpath = '/cpacs/vehicles/aircraft/model/analyses/massBreakdown/fuel/fuelInTanks/massDescription/mass'
M_payload_xpath = '/cpacs/vehicles/aircraft/model/analyses/massBreakdown/payload/massDescription/mass'
M_oew_xpath = '/cpacs/vehicles/aircraft/model/analyses/massBreakdown/mOEM/massDescription/mass'

In [ ]:
# Get Fuel Weight, W_fuel
M_fuel = tixi_h.getDoubleElement(M_fuel_xpath)
W_fuel = M_fuel * 9.81 # N
print('Fuel Weight: ', W_fuel, 'N')

In [ ]:
# Get Payload Weight, W_payload
# ...

In [ ]:
# %load ./solutions/tixiGetPayloadWeight.py

In [ ]:
# Get OEW Weight, W_oew
# ...

In [ ]:
# %load ./solutions/tixiGetOewWeight.py

## Step 3: Calculate the Design Range

Now that we have all the data necessary for the range equation, we can use it to **calculate the range of the aircraft**.  

In the CPACS file, two design ranges are defined:

- **Required range**: The target range specified in the design requirements  
- **Actual range**: The calculated range based on aircraft performance parameters  

Our task is to calculate the actual range using the Breguet equation and then update the **`<actual>`** value in the CPACS file. This allows us to directly compare the **required** and **achieved** performance in the digital model.

![Referencing the design ranges](resources/cpacs_updateDesignRange.png)

In [ ]:
### Calculate Breguet Range
import math

def calculate_range(V_cruise, TSFC, L_D, W_fuel, W_payload, W_oew):
    """Calculate range using Breguet equation"""
    fuel_fraction = W_fuel / (W_payload + W_oew)
    R = (V_cruise / TSFC) * L_D * math.log(1 + fuel_fraction)
    return R

# Calculate current range
R = calculate_range(V_cruise, TSFC, L_D, W_fuel, W_payload, W_oew)

# convert to km and round
R_km = round(R / 1000, 0) 
print('Calculated Range:', R_km, 'km')

## Step 4: Validate the result
Lets read the design range for this aircraft and compare the result.

In [ ]:
# Read Current actual Design Range
R_actual = tixi_h.getDoubleElement('/cpacs/vehicles/aircraft/model/global/designRange/actual')

# convert to km and round
R_actual_km = round(R_actual / 1000, 0) 
print('Design Actual Range:', R_actual_km, 'km')

## Step 5: Write Data to a CPACS Model

After we have seen how to read data from a CPACS model, let us now write data to it!

Let's update the actual range of the CPACS file the calculated range:

In [ ]:
# Actual Range Xpath
actual_range_xpath = '/cpacs/vehicles/aircraft/model/global/designRange/actual'

Now we need to update the **actual range** in the CPACS file using the `updateDoubleElement()` command from TiXI.  

For example, if we wanted to update the element `cargoCapacity` with a new value stored in `new_cargoCapacity`, the command would look like this:  

```python
tixi_h.updateDoubleElement(cargoCapacity_xpath, float(new_cargoCapacity), '%.1f')

In [ ]:
# Update the actual range
# ...

In [ ]:
# %load ./solutions/tixiUpdateActualRange.py

## Step 6: Save a CPACS File

In [ ]:
# save and close the file
# ...

In [ ]:
# %load ./solutions/tixiSaveFile.py

Don't forget to call the `tixi_h.close()` method after saving!

You can now also open that CPACS file with a text editor and see your modifications.

## Module 2: Study the Change of Range According to Fuel Weight Change

In this module, we will examine how variations in fuel weight affect the aircraft’s range.

In [ ]:
tixi_h.open('cpacs_files/Exact_turbofan.xml')

In [ ]:
# Test different fuel weight changes
# (+) for increase, (-) for decrease
fuel_change = 0.2 # 20% 

In [ ]:
# Calculate new fuel weight
W_fuel_new = W_fuel + W_fuel * fuel_change
print('New Fuel Weight: ', W_fuel_new, 'N')

Lets check how that changes our range

In [ ]:
# Calculate range with updated fuel weight
R_new = calculate_range(V_cruise, TSFC, L_D, W_fuel_new, W_payload, W_oew)

# convert to km and round
R_new_km = round(R_new / 1000, 0) 
print('Range:', R_new_km, 'km')

# Calculate the change of range by percentage
R_change = (R_new_km - R_km) / R_km * 100   # use old range as reference

if R_change > 0:
    print(f"Range increased by {R_change:.2f}%")
elif R_change < 0:
    print(f"Range decreased by {abs(R_change):.2f}%")
else:
    print("No change in range.")

## Write Data to a CPACS Model

After we have seen how to read data from a CPACS model, let us now write data to it!

Let's add the calculated range to the CPACS file in the place of the actual range: 

In [ ]:
# Update the actual range
# ...

In [ ]:
# %load ./solutions/tixiUpdateActualRangeWithNewRange.py

## Save a CPACS File

In [ ]:
# save and close the file
# ...

In [ ]:
# %load ./solutions/tixiSaveFileNewRange.py

Don't forget to call the `tixi_h.close()` method after saving!

You can now also open that CPACS file with a text editor and see your modifications.